This here is to keep main ideas while I go through [neuralnetworksanddeeplearning.com](http://neuralnetworksanddeeplearning.com) book, real quick:

* **perceptrons**: 

    * a **step function**
    * by varying the **bias**(-threshold) we can achieve different set of decision making, regardless of weights.
    * high(positive) bias means that it's very easy to make perceptron to fire, and vice versa:
    $\begin{eqnarray}
  \mbox{output} = \left\{ 
    \begin{array}{ll} 
      0 & \mbox{if } w\cdot x + b \leq 0 \\
      1 & \mbox{if } w\cdot x + b > 0
    \end{array}
  \right.
\end{eqnarray}$

    * you can use perceptrons to simulate a NAND gate, they are **unviersal for computation**, hence perceptrons are also universal for computation. you can use them to simulate any function! they are as good as any computing device. this is hardly a big news!
    * BUT you can device *learning algorithms* that would lay out this "circuts of gates"(neural net with appropriate bias and weights) for you. this makes the difference.
    * what would make learning possible(weights and bias) for the network is the property that small change in the weights and bias makes a small change in the output of the net.
    * for a step function this is not possible, you need a smooth function **activation function**, hence **sigmoid** function.
* **sigmoid neurons**:
   
    * sigmoid function $\sigma(z)=\frac{1}{1+\exp(-z)}$
    * can take real values instead of 1 and 0 for input and output
    
* **feed forward networks**: information is always fed forward, there is no loop. otherwise output will depend on the input. if there is loop it is called **recurrent neural network**, but looping is not instant, the ouput will be fed to input at some later time. 

* to determine the input layer dimension we look at the data dimesnion, e.g. for a $64\times64$ grey scale image, the input layer is 4096 dimension with each value being the intensity of the pixel.

* **segmentation problem** break up a sequence of numbers to stand alone numbers(images)
* **cost function(loss function, objective function)**: an example: 
$\begin{eqnarray}  C(w,b) \equiv
  \frac{1}{2n} \sum_x \| y(x) - a\|^2.
\end{eqnarray}$ *quadratic cost function* or *mean squared error*
* **gradient descent** an algorithm to minimize cost function. you always want to have cost fucntion decreasing, so, $C:=C(\vec\nu), \Rightarrow \Delta C = \nabla C\cdot\Delta\nu$ so if we choose, $\Delta\nu = -\eta\nabla C$, $\eta$ is **learning rate**. we will use this equation to update new positons(values), $\nu^\prime = \nu - \eta\nabla C$, if learning rate is too big we have a problem with approximation, we may end up increasing cost function, if it's too small, it will take too much time to find the minimum. gradient descent update rule:
$ w_k \rightarrow w_k' = w_k-\eta \frac{\partial C}{\partial w_k} $ and  $ b_l \rightarrow b_l' = b_l-\eta \frac{\partial C}{\partial b_l} $

* cost function is an average over cost function for individual cost function over each training example. computing gradient over large training dataset would take much more time. hence **stochastic gradient descent**, the idea is to sample a random mini-batch over training example and use: 
$ \begin{eqnarray}
  \frac{\sum_{j=1}^m \nabla C_{X_{j}}}{m} \approx \frac{\sum_x \nabla C_x}{n} = \nabla C,
\end{eqnarray}$

* stochastic gradient descent works by picking out a randomly chosen mini-batch of training inputs, and training with those. Then we pick out another randomly chosen mini-batch and train with those. And so on, until we've exhausted the training inputs, which is said to complete an epoch of training. At that point we start over with a new training epoch. if the batch size is one(like humans), then it is called *online* or *on-line* or incremental learning.

* people use *validation* data to set *hyper-parameters* of the model, learning rate, ...

* Main equations of backpropagation:

$\begin{eqnarray} 
  a^{l}_j = \sigma\left( \sum_k w^{l}_{jk} a^{l-1}_k + b^l_j \right),
\tag{1}\end{eqnarray}$

activation
$\begin{eqnarray} 
  a^{l} = \sigma(w^l a^{l-1}+b^l).
\tag{2}\end{eqnarray}$

weighted ouput
$\begin{eqnarray}
z^l_j = \sum_k w^l_{jk} a^{l-1}_k+b^l_j
\tag{3}\end{eqnarray}$

the error in node j of layer l
$\begin{eqnarray} 
  \delta^l_j \equiv \frac{\partial C}{\partial z^l_j}.
\tag{4}\end{eqnarray}$

error in the output layer
$\begin{eqnarray} 
  \delta^L_j = \frac{\partial C}{\partial a^L_j} \sigma'(z^L_j).
\tag{BP1}\end{eqnarray}$

error in layer $l$ based on error in layer $l+1$
$\begin{eqnarray} 
  \delta^l = ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l),
\tag{BP2}\end{eqnarray}$
By combining (BP2) with (BP1) we can compute the error δl for any layer in the network. We start by using (BP1) to compute δL, then apply Equation (BP2) to compute δL−1, then Equation (BP2) again to compute δL−2, and so on, all the way **back** through the network.

$\begin{eqnarray}  \frac{\partial C}{\partial b^l_j} =
  \delta^l_j.
\tag{BP3}\end{eqnarray}$

$\begin{eqnarray}
  \frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k \delta^l_j.
\tag{BP4}\end{eqnarray}$

* see the backpropagation algorithm, pseudocode: [link](http://neuralnetworksanddeeplearning.com/chap2.html#the_backpropagation_algorithm)

In [76]:
import random
import numpy as np
class Network(object):
    def __init__(self, sizes):
        """sizes is a list of layers, each element corresponds to the size of the layer."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(x, 1) for x in sizes[1:]]
        self.weights = [np.random.randn(x, y) for x, y in zip(sizes[1:], sizes[:-1])]
        
    def feedforward(self, a):
        for w, b in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w, a) + b)
            print(a)
        return a
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
        
    def SGD(self, training_data, batch_size, epochs, eta, test_data=None):
        """stochastic gradient descent algorithm. randomly select a mini batch from
        training data, learn from it, do this again until all training data is exausted,
        repeat this for epoch times. the training data is a list of tuples `(x, y)`.
        y being the labels"""
        test_data = list(test_data)
        if test_data:
            n_test = len(test_data)
        training_data = list(training_data)
        n = len(training_data)
        for i in range(epochs):
            random.shuffle(training_data)
            batches = [training_data[k:k+batch_size] for k in np.random.randn(0, n, batch_size)]
            for batch in batches:
                self.update_batch(batch, eta)
            if test_data:
                print("Epoch: {}, {}/{} ".format(i, self.evaluate(test_data), n_test))
            else:
                print("Epoch: {} completed.".format(i))
                
        def update_batch(self, batch, eta):
            nabla_b = [np.zeros(np.shape(b)) for b in self.biases]
            nabla_w = [np.zeros(np.shape(w)) for w in self.weights]
            for x, y in batch:
                delta_nabla_b, delta_nabla_w = self.backprop(x, y)
                nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
                nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            self.biases = [b-(eta/batch_size)*nb for b, nb in zip(self.biases, delta_nabla_b)]
            self.weights = [w-(eta/batch_size)*nw for w,nw in zip(self.weights, delta_nabla_w)]
        
        def backprop(self, x, y):
            nabla_b = [np.zeros(b.shape) for b in self.biases]
            nabla_w = [np.zeros(w.shape) for w in self.biases]
            activation = x
            activations = [x] 
            zs = [] 
            #forward pass
            for b, w in zip(self.biases, self.weights):
                z = np.dot(w, activation)+b
                zs.append(z)
                activation = sigmoid(z)
                activations.append(activation)
            #backward pass
            delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
            nabla_b[-1] = delta
            nabla_w[-1] = np.dot(delta, activations[-2].transpose())
            
            for l in xrange(2, self.num_layers):
                z = zs[-l]
                sp = sigmoid_prime(z)
                delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
                nabla_b[-l] = delta
                nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (nabla_b, nabla_w)

        def cost_derivative(self, output_activations, y):
             return (output_activations-y)

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [54]:
import gzip

def fetch(path):
    with open(path, "rb") as f:
        dat = f.read()
    return np.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()
X_train = fetch("data/mnist/train-images-idx3-ubyte.gz")[0x10:]#.reshape((-1, 28, 28))
Y_train = fetch("data/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("data/mnist/t10k-images-idx3-ubyte.gz")[0x10:]#.reshape((-1, 28, 28))
Y_test = fetch("data/mnist/t10k-labels-idx1-ubyte.gz")[8:]

import matplotlib.pyplot as plt
#plt.imshow(X_test[10])

In [34]:
#training_data = [(x, y) for x, y in zip(X_train, Y_train)]
#test_data = [(x, y) for x, y in zip(X_test, Y_test)]

In [64]:
import pickle
def load_data():
    f = gzip.open('./data/mnist/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [70]:
training_data, validation_data, test_data = load_data_wrapper()

In [77]:
nn = Network([784, 30, 10])
nn.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch: 0 completed.
Epoch: 1 completed.
Epoch: 2 completed.
Epoch: 3 completed.
Epoch: 4 completed.
Epoch: 5 completed.
Epoch: 6 completed.
Epoch: 7 completed.
Epoch: 8 completed.
Epoch: 9 completed.


NameError: name 'nabla_b' is not defined

---
---

* cross entropy cost function:
$\begin{eqnarray} 
  C = -\frac{1}{n} \sum_x \left[y \ln a + (1-y ) \ln (1-a) \right],
\end{eqnarray}$
* softmax layer
$\begin{eqnarray} 
  a^L_j = \frac{e^{z^L_j}}{\sum_k e^{z^L_k}},
\end{eqnarray}$ $z^L_j = \sum_{k} w^L_{jk} a^{L-1}_k + b^L_j$